In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

def make_bins(xlim,dx):
    Edges_x = np.arange(xlim[0],xlim[1]+dx,dx)
    Bins_x = Edges_x[:-1]+dx/2
    return Edges_x,Bins_x

def ind_match(x,A):
    return np.argwhere(A == x)[:,0][0]

In [3]:
path = '../DataFrames/df_road_v3_zeros.csv'
df_road = pd.read_csv(path)

In [4]:
Ys = 2005 #must start before or on this year
Ye = 2018 #must end this year or later

Igap = df_road.Max_year_gap == 1 #we don't want gaps
Istart = df_road.Year_start <= Ys
Iend = df_road.Year_end >= Ye
Iyears = Igap & Istart & Iend

df_road_sub = df_road.loc[Iyears,:].copy()

In [8]:
f_include = ['ID', 'total', 'slight', 'serious',
       'bikelane','aadf_Road_name', 'aadf_Road_category', 'aadf_Road_type',
       'aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME', 'aadf_Link_length_miles',
       'aadf_Pedal_cycles', 'aadf_All_motor_vehicles',
       'aadf_All_HGVs', 'aadf_LGVs', 'aadf_Buses_and_coaches', 'total_dnorm',
       'total_dnorm_bvnorm', 'serious_ratio']

df_cluster = df_road_sub[f_include].copy()

#make a sample length scaled total normalized to aadf_Link_length instead
LL_med = df_cluster.aadf_Link_length_miles.median()
df_cluster['total_dnorm_LL'] = df_cluster.total.values*LL_med/df_cluster.aadf_Link_length_miles.values

In [9]:
f_scale = ['aadf_Pedal_cycles', 'aadf_All_motor_vehicles',
       'aadf_All_HGVs', 'aadf_LGVs', 'aadf_Buses_and_coaches']

for f in f_scale:
    fname = f+'_s'
    mean = df_cluster[f].mean()
    std = df_cluster[f].std()
    #df_cluster.loc[:,f] = (df_cluster[f].values - mean)/std
    df_cluster[fname] = (df_cluster[f].values - mean)/std

In [10]:
#pick which features to try clustering on
f_cluster = ['bikelane','aadf_Pedal_cycles_s', 'aadf_All_motor_vehicles_s', 'aadf_All_HGVs_s',
       'aadf_LGVs_s', 'aadf_Buses_and_coaches_s']

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

In [20]:
import os
os.environ["OMP_NUM_THREADS"] = '9'

In [25]:
KM = KMeans(3)

KM.fit(df_cluster[f_cluster])
cluster_centers = KM.cluster_centers_

inertias = KM.inertia_

clusters = KM.predict(df_cluster[f_cluster])

C:\Users\tuthill.15\Anaconda3\envs\geo_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.
  warnings.warn(


In [28]:
np.unique(clusters, return_counts=True)

(array([0, 1, 2]), array([1610,  169,  340], dtype=int64))